In [1]:
# from sentence_transformers import SentenceTransformer
# import polars as pl

import pandas as pd
import faiss  
import numpy as np
import gc
import pickle
import os
from tqdm import tqdm
from os.path import join as path_join

In [35]:
# model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

In [2]:
path = "../data/raw/hh_recsys_vacancies.pq"
embedding_root = "dumps/production/i2i/name_area"
vacancies_tmp_path = path_join(embedding_root, "vacancies.parquet")
embeddings_path = path_join(embedding_root, "embeddings.pickle")

# shards_root    = "dumps/production/i2i/paraphrase_multilingual_MiniLM_L12_v2_384d/shards"
# indexes_root   = "dumps/production/i2i/paraphrase_multilingual_MiniLM_L12_v2_384d/indexes"
# output_path = "dumps/production/i2i/paraphrase_multilingual_MiniLM_L12_v2_384d.parquet"

In [3]:
if not os.path.exists(embedding_root):
    os.makedirs(embedding_root)

# if not os.path.exists(shards_root):
#     os.makedirs(shards_root)

# if not os.path.exists(indexes_root):
#     os.makedirs(indexes_root)

In [4]:
# vacancies = pl.read_parquet(path).to_pandas()

In [5]:
# vacancies.head(1)

,vacancy_id,name,company.id,description,keySkills.keySkill,compensation.from,compensation.to,compensation.currencyCode,area.id,area.regionId,employment,workSchedule,workExperience
0,v_862116,Смотритель музейный,c_162972,<strong>Обязанности:</strong> <ul> <li>Осущест...,"[Пользователь ПК, Работа в команде, Умение пла...",16500.0,NaN,RUR,a_4761,ar_33,full,fullDay,noExperience


In [6]:
# vacancies.iloc[3]["description"]

'<p><strong>Для каждого, кто хочет работать и зарабатывать! Для каждого, кто готов к профессиональному и карьерному росту! Ведущая федеральная розничная сеть магазинов «Пятерочка» приглашает на работу:</strong></p> <p> </p> <p><strong>ЧЕМ НУЖНО ЗАНИМАТЬСЯ:</strong></p> <ul> <li>Выпечка хлебобулочных изделий из полуфабрикатов</li> <li>Упаковка, фасовка, маркировка</li> <li>Обеспечивать наполненность витрины</li> </ul> <p> </p> <p><strong>ОТ ВАС:</strong></p> <ul> <li>Желание работать с хлебобулочными продуктами, остальному научим!</li> </ul> <p> </p> <p><strong>ОТ НАС:</strong></p> <ul> <li>Оформление по ТК РФ</li> <li>Фиксированный оклад + премии и надбавки за стаж. Средний доход 38500 – 42000 руб. в месяц до вычета налогов</li> <li>График работы 5/2, 2/2, возможен неполный рабочий день, неполная рабочая неделя</li> <li>Финансовая поддержка в сложных жизненных ситуациях</li> <li>Медицинская книжка за счет компании</li> <li>Скидки в магазинах сети, скидки от партнеров</li> <li>Оборудова

In [7]:
# vacancies["area.regionId"].value_counts()

# area_data = []

# for area_id, cnt in vacancies["area.regionId"].value_counts().items():
#     area_data.append((cnt, area_id))

# area_data = sorted(area_data, reverse=True)

# area_data[:3], len(area_data)

([(601764, 'ar_41'), (228421, 'ar_78'), (173943, 'ar_64')], 105)

In [8]:
# cities = """
# Алтайский край 16221
# Амурская область 5113
# Архангельская область 5539
# Астраханская область 4793
# Белгородская область 10439
# Брянская область 6272
# Владимирская область 11966
# Волгоградская область 14595
# Вологодская область 8207
# Воронежская область 21711
# Донецкая Народная Республика 813
# Еврейская АО 558
# Забайкальский край 4870
# Запорожская область 178
# Ивановская область 8100
# Иркутская область 20369
# Кабардино-Балкарская республика 778
# Калининградская область 8111
# Калужская область 9966
# Камчатский край 1291
# Карачаево-Черкесская Республика 589
# Кемеровская область 22103
# Кировская область 8289
# Костромская область 4796
# Краснодарский край 55067
# Красноярский край 28476
# Курганская область 4795
# Курская область 6255
# Ленинградская область 15119
# Липецкая область 8748
# Луганская Народная Республика 476
# Магаданская область 1104
# Москва 254274
# Московская область 96015
# Мурманская область 5706
# Ненецкий АО 106
# Нижегородская область 34216
# Новгородская область 3827
# Новосибирская область 37885
# Омская область 16231
# Оренбургская область 8896
# Орловская область 4587
# Пензенская область 7641
# Пермский край 21205
# Приморский край 13664
# Псковская область 3426
# Республика Адыгея 1540
# Республика Алтай 771
# Республика Башкортостан 29152
# Республика Бурятия 4572
# Республика Дагестан 2485
# Республика Ингушетия 172
# Республика Калмыкия 507
# Республика Карелия 3243
# Республика Коми 3120
# Республика Крым 8956
# Республика Марий Эл 3345
# Республика Мордовия 3605
# Республика Саха (Якутия) 4030
# Республика Северная Осетия-Алания 1159
# Республика Татарстан 41594
# Республика Тыва 319
# Республика Хакасия 2527
# Ростовская область 33253
# Рязанская область 9764
# Самарская область 33074
# Санкт-Петербург 107511
# Саратовская область 14324
# Сахалинская область 3423
# Свердловская область 58932
# Смоленская область 5441
# Ставропольский край 11953
# Тамбовская область 5171
# Тверская область 10884
# Томская область 10834
# Тульская область 14320
# Тюменская область 18822
# Удмуртская Республика 11850
# Ульяновская область 7768
# Хабаровский край 13600
# Ханты-Мансийский АО - Югра 12103
# Херсонская область 62
# Челябинская область 35943
# Чеченская республика 505
# Чувашская Республика 7840
# Чукотский АО 120
# Ямало-Ненецкий АО 4001
# Ярославская область 12559
# """

# cities_data = []

# for city in cities.split("\n"):
#     if city != "":
#         city = city.strip()
#         city_name = " ".join(city.split(" ")[:-1])
#         city_count = int(city.split(" ")[-1])
#         cities_data.append((city_count, city_name))

# cities_data = sorted(cities_data, reverse=True)

# cities_data[:3], len(cities_data)

([(254274, 'Москва'),
  (107511, 'Санкт-Петербург'),
  (96015, 'Московская область')],
 88)

In [4]:
manual_parsed_regions = """
ar_41 Москва
ar_78 Санкт-Петербург
ar_64 Московская область
ar_21 Красноярский край
ar_44 Свердловская область
ar_71 Республика Татарстан
ar_52 Новосибирская область
ar_77 Нижегородской область
ar_99 Ростовская область
ar_97 Челябинской области
ar_2 Самарская область
ar_39 Республика Башкортостан
ar_20 Красноярский край
ar_8 Воронежская область
ar_72 Пермский край
ar_11 Кемеровской область
ar_23 Приморская область
ar_91 Иркутская область
ar_56 Тюменская область
ar_42 Хабаровский край
ar_1 Омская область
ar_87 Алтайский край
ar_94 Волгоградская область
ar_51 Ленинградская область
ar_28 Саратовская область
ar_102 Тульская область
ar_69 Ярославская область
ar_96 Ставропольский край
ar_104 Ханты-Мансийский автономный округ - Югра
ar_59 Удмуртская Республика
ar_25 Владимирская область
ar_66 Томская область
ar_3 Тверская область
ar_65 Белгородская область
ar_92 Калужская область
ar_36 Оренбургская область
ar_0 Рязанская область
ar_13 Республика Крым
ar_74 Липецкая область
ar_38 Кировская область
ar_61 Калининградская область
ar_37 Чувашская Республика
ar_7 Вологодская область
ar_79 Пензенская область
ar_16 Ульяновская область
ar_47 Ивановская область
ar_55 Курская область
ar_32 Брянская область
ar_58 Мурманская область
ar_45 Архангельская область
ar_22 Смоленская область
ar_31 Тамбовская область
ar_100 Амурская область
ar_70 Астраханская область
ar_40 Забайкальский край
ar_103 Костромская область
ar_48 Орловская область
ar_101 Курганская область
ar_84 Ямало-Ненецкий автономный округ
ar_18 Республика Бурятия
ar_73 Новгородская область
ar_68 Сахалинская область
ar_90 Республика Саха (Якутия)
ar_82 Республика Коми
ar_76 Республика Мордовия
ar_81 Республика Марий Эл
ar_49 Республика Карелия
ar_9 Псковская область
ar_4 Республика Хакасия
ar_95 Республика Дагестан
ar_60 Республика Адыгея
ar_89 Камчатский край
ar_10 Республика Северная Осетия-Алания
ar_67 Магаданская область
ar_63 Республика Алтай
ar_15 Донецкая Народная Республика
ar_12 Кабардино-Балкарская республика
ar_19 Карачаево-Черкесская Республика
ar_33 Чеченская республика
ar_5 Республика Калмыкия
ar_85 Луганская Народная Республика
ar_6 Еврейская автономный округ
ar_62 Республика Тыва
ar_24 Республика Ингушетия
ar_88 Запорожская область
ar_17 Чукотский автономный округ
ar_26 Ненецкий АО
ar_54 Херсонская область
"""

region_2_name = {
    row.strip().split(" ")[0]: " ".join(row.strip().split(" ")[1:])
        for row in filter(lambda x: x.strip() != "", manual_parsed_regions.split("\n"))
}
region_2_name['ar_54']

'Херсонская область'

In [31]:
# for i, j in zip(area_data, cities_data):
#     print(i, j)

In [27]:
# vacancies_not_parsed = vacancies[~vacancies["area.regionId"].isin(set(region_2_name.keys()))]

In [32]:
# for area in area_data:
    
    
#     vacancies_area = vacancies_not_parsed[["name", "area.regionId"]][vacancies_not_parsed["area.regionId"] == area[1]]

#     if vacancies_area.shape[0] == 0:
#         continue

#     print("-=-=-=-=-=-=-=-=-=-=-=-=-=-=-")

#     vacancies_area_city = vacancies_area[vacancies_area["name"].apply(lambda name: "г." in name)]
#     vacancies_area_city = vacancies_area_city .sample(n=min(vacancies_area_city.shape[0], 20))
#     for row_index, row in vacancies_area_city.iterrows():
#         print(row["area.regionId"], row["name"])
    
#     vacancies_area = vacancies_area.sample(n=min(vacancies_area.shape[0], 20))
#     for row_index, row in vacancies_area.iterrows():
#         print(row["area.regionId"], row["name"])

In [33]:
# for x in cities_data[60:]:
#     print(x[1])

In [34]:
# custom_vacancies = vacancies[vacancies["area.regionId"].isin(set(region_2_name.keys()))].sample(n=50000)

In [39]:
# to_encode = custom_vacancies[["vacancy_id", "area.regionId", "name"]]
# to_encode["region_name"] = custom_vacancies["area.regionId"].map(region_2_name)
# to_encode["text"] = to_encode["region_name"] + ". " + to_encode["name"]
# to_encode = to_encode[["vacancy_id", "text"]].dropna()

/var/folders/dz/7q462fh56rg95y4_6lnqmrvr0000gq/T/ipykernel_5237/1005022061.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  to_encode["region_name"] = custom_vacancies["area.regionId"].map(region_2_name)
/var/folders/dz/7q462fh56rg95y4_6lnqmrvr0000gq/T/ipykernel_5237/1005022061.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  to_encode["text"] = to_encode["region_name"] + ". " + to_encode["name"]


In [40]:
# to_encode_vacancy_id = to_encode["vacancy_id"].to_list()
# to_encode_text       = to_encode["text"].to_list()

In [41]:
# embeddings = model.encode(to_encode_text, show_progress_bar=True, batch_size=256)

Batches: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 196/196 [01:13<00:00,  2.66it/s]


In [27]:
# embeddings = embeddings / np.linalg.norm(embeddings, axis = 1, keepdims=True)

In [42]:
# index = faiss.IndexFlatL2(embeddings[0].shape[0])
# index.add(embeddings)
# print(index.ntotal)

50000


In [5]:
# for i in range(10):
#     idx = np.random.randint(0, len(embeddings))

#     print("-=-=-=-=-=-=-")

#     vacancy = custom_vacancies.iloc[idx]

#     vacancy_id = to_encode_vacancy_id[idx]
#     text       = to_encode_text[idx]
#     embedding  = np.expand_dims(embeddings[idx], axis=0)
#     D, I = index.search(embedding, 10)

#     print(text)
#     print("  - keySkills.keySkill:", vacancy["keySkills.keySkill"])

#     print("^^^^^^^^^^^^^")

#     for distances, indexes in zip(D[0], I[0]):
#         neighbour_vacancy = custom_vacancies.iloc[idx]
#         # print("    - keySkills.keySkill:", vacancy["keySkills.keySkill"])
        
#         print(to_encode_text[indexes])

In [44]:
# Теперь 

vacancies_used = vacancies[vacancies["area.regionId"].isin(set(region_2_name.keys()))]
vacancies_used = vacancies_used[["vacancy_id", "area.regionId", "name"]]
vacancies_used["region_name"] = vacancies_used["area.regionId"].map(region_2_name)
vacancies_used["text"] = vacancies_used["region_name"] + ". " + vacancies_used["name"]
vacancies_used = vacancies_used[["vacancy_id", "text"]].dropna()

vacancies_used_vacancy_ids = vacancies_used["vacancy_id"].to_list()
vacancies_used_names       = vacancies_used["text"].to_list()

In [45]:
vacancies_used.shape

(2712385, 2)

In [46]:
embeddings_used = model.encode(vacancies_used_names, show_progress_bar=True, batch_size=256)

Batches: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10596/10596 [1:02:33<00:00,  2.82it/s]


In [47]:
vacancies_used.rename(columns={"text": "name"}).to_parquet(vacancies_tmp_path)
pickle.dump(embeddings_used, open(embeddings_path, 'wb+'), protocol=4)

In [48]:
del vacancies
del vacancies_used
del embeddings_used
gc.collect()

0

In [6]:
"""
    Make indexes
"""

indexes_root = path_join(embedding_root, "indexes")

embeddings = pickle.load(open(embeddings_path, 'rb'))
embeddings = embeddings / np.linalg.norm(embeddings, axis = 1, keepdims=True)

total_n = len(embeddings)
max_index_size = 256000
indexes_n = (total_n // max_index_size) + (0 if total_n % max_index_size == 0 else 1)

del embeddings
gc.collect()

for path in list(map(lambda x: path_join(indexes_root, x), os.listdir(indexes_root))):
    os.remove(path)
 
for i in tqdm(range(indexes_n), total=indexes_n):
    idx_from = max_index_size * i
    idx_to   = max_index_size * i + max_index_size

    embeddings = pickle.load(open(embeddings_path, 'rb'))[idx_from:idx_to]
    embeddings = embeddings / np.linalg.norm(embeddings, axis = 1, keepdims=True)
    embeddings = np.float32(embeddings)

    index = faiss.IndexHNSWFlat(embeddings[0].shape[0], 32, faiss.METRIC_INNER_PRODUCT)
    index.add(embeddings)
    print(index.ntotal)

    index_path = path_join(indexes_root, f"index_{idx_from}_{idx_to}")
    faiss.write_index(index, index_path)

    del index
    del embeddings
    gc.collect()

  0%|                                                                                                                                                                                                       | 0/11 [00:00<?, ?it/s]

256000


  9%|█████████████████▎                                                                                                                                                                             | 1/11 [00:15<02:33, 15.39s/it]

256000


 18%|██████████████████████████████████▋                                                                                                                                                            | 2/11 [00:31<02:21, 15.70s/it]

...

 82%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎                                  | 9/11 [02:26<00:33, 16.62s/it]

256000


 91%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋                 | 10/11 [02:42<00:16, 16.52s/it]

152385


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 11/11 [02:51<00:00, 15.62s/it]


In [13]:
indexes_pathes = list(map(lambda x: str(path_join(indexes_root, x)), os.listdir(indexes_root)))
for i in indexes_pathes:
    print(i)

dumps/production/i2i/name_area/indexes/index_512000_768000
dumps/production/i2i/name_area/indexes/index_1024000_1280000
dumps/production/i2i/name_area/indexes/index_256000_512000
dumps/production/i2i/name_area/indexes/index_1536000_1792000
dumps/production/i2i/name_area/indexes/index_1280000_1536000
dumps/production/i2i/name_area/indexes/index_2048000_2304000
dumps/production/i2i/name_area/indexes/index_768000_1024000
dumps/production/i2i/name_area/indexes/index_2560000_2816000
dumps/production/i2i/name_area/indexes/index_1792000_2048000
dumps/production/i2i/name_area/indexes/index_0_256000
dumps/production/i2i/name_area/indexes/index_2304000_2560000


In [19]:
shards_root = path_join(embedding_root, "shards")

embeddings = pickle.load(open(embeddings_path, 'rb'))
embeddings = embeddings / np.linalg.norm(embeddings, axis = 1, keepdims=True)

total_n = len(embeddings)
max_i2i_in_shard = 256000
shards_n = (total_n // max_i2i_in_shard) + (0 if total_n % max_i2i_in_shard == 0 else 1)
neighbours_n = 101 # Ибо одна из вершин будет собственным эмбеддингом

del embeddings
gc.collect()

# for path in list(map(lambda x: path_join(shards_root, x), os.listdir(shards_root))):
#     os.remove(path)

output_pathes = []

indexes_paths = list(map(
    lambda x: x[1],
    sorted(list(map(
        lambda x: (int(x.split("_")[-2]), path_join(indexes_root, x)), os.listdir(indexes_root)
    )))
))

for i in indexes_paths:
    print(i)

for i in tqdm(range(shards_n), total=shards_n):
    # Сколько взять для поиска?

    if i >= 2:
        continue

    idx_from = max_i2i_in_shard * i
    idx_to   = max_i2i_in_shard * i + max_i2i_in_shard

    embeddings = pickle.load(open(embeddings_path, 'rb'))[idx_from:idx_to]
    embeddings = embeddings / np.linalg.norm(embeddings, axis = 1, keepdims=True)
    embeddings = np.float32(embeddings)

    output = [[] for i in range(len(embeddings))]

    for index_path in indexes_paths:
        print(index_path)
        print(int(index_path.split("_")[-2]))

        idx_start = int(index_path.split("_")[-2])
        
        index = faiss.read_index(index_path)
        D, I = index.search(embeddings, neighbours_n)

        for j, (dists, idxs) in enumerate(zip(D, I)):
            output[j] = sorted(
                output[j] + list(zip(dists, map(lambda x: int(x + idx_start), idxs))),
                reverse=True)[:neighbours_n]

        del index
        gc.collect()

    # # Сколько по итогу взяли для поиска
    output_idx_from = max_i2i_in_shard * i
    output_idx_to = max_i2i_in_shard * i + len(embeddings)

    distances = list(map(lambda neigbours: list(map(lambda x: x[0], neigbours)), output))
    neighbour_indexes = list(map(lambda neigbours: list(map(lambda x: x[1], neigbours)), output))

    shard_output_path = path_join(shards_root, f"shard_{output_idx_from}_{output_idx_to}")
    df = pd.DataFrame.from_dict({
        "idx": list(range(output_idx_from, output_idx_to)),
        "distances": distances,
        "neighbour_idxs": neighbour_indexes})
    df.to_parquet(shard_output_path)

    output_pathes.append(shard_output_path)

"\n".join(output_pathes)

dumps/production/i2i/name_area/indexes/index_0_256000
dumps/production/i2i/name_area/indexes/index_256000_512000
dumps/production/i2i/name_area/indexes/index_512000_768000
dumps/production/i2i/name_area/indexes/index_768000_1024000
dumps/production/i2i/name_area/indexes/index_1024000_1280000
dumps/production/i2i/name_area/indexes/index_1280000_1536000
dumps/production/i2i/name_area/indexes/index_1536000_1792000
dumps/production/i2i/name_area/indexes/index_1792000_2048000
dumps/production/i2i/name_area/indexes/index_2048000_2304000
dumps/production/i2i/name_area/indexes/index_2304000_2560000
dumps/production/i2i/name_area/indexes/index_2560000_2816000


  0%|                                                                                                                                                                                                       | 0/11 [00:00<?, ?it/s]

dumps/production/i2i/name_area/indexes/index_0_256000
0
dumps/production/i2i/name_area/indexes/index_256000_512000
256000
dumps/production/i2i/name_area/indexes/index_512000_768000
512000
dumps/production/i2i/name_area/indexes/index_768000_1024000
768000
dumps/production/i2i/name_area/indexes/index_1024000_1280000
1024000
dumps/production/i2i/name_area/indexes/index_1280000_1536000
1280000
dumps/production/i2i/name_area/indexes/index_1536000_1792000
1536000
dumps/production/i2i/name_area/indexes/index_1792000_2048000
1792000
dumps/production/i2i/name_area/indexes/index_2048000_2304000
2048000
dumps/production/i2i/name_area/indexes/index_2304000_2560000
2304000
dumps/production/i2i/name_area/indexes/index_2560000_2816000
2560000


  9%|█████████████████▎                                                                                                                                                                            | 1/11 [04:34<45:43, 274.37s/it]

dumps/production/i2i/name_area/indexes/index_0_256000
0
dumps/production/i2i/name_area/indexes/index_256000_512000
256000
dumps/production/i2i/name_area/indexes/index_512000_768000
512000
dumps/production/i2i/name_area/indexes/index_768000_1024000
768000
dumps/production/i2i/name_area/indexes/index_1024000_1280000
1024000
dumps/production/i2i/name_area/indexes/index_1280000_1536000
1280000
dumps/production/i2i/name_area/indexes/index_1536000_1792000
1536000
dumps/production/i2i/name_area/indexes/index_1792000_2048000
1792000
dumps/production/i2i/name_area/indexes/index_2048000_2304000
2048000
dumps/production/i2i/name_area/indexes/index_2304000_2560000
2304000
dumps/production/i2i/name_area/indexes/index_2560000_2816000
2560000


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 11/11 [09:03<00:00, 49.44s/it]


'dumps/production/i2i/name_area/shards/shard_0_256000\ndumps/production/i2i/name_area/shards/shard_256000_512000'

In [20]:
# i2i_pathes = list(map(lambda x: path_join(shards_root, x), os.listdir(shards_root)))
# print(i2i_pathes)

result = pd.read_parquet("dumps/production/i2i/name_area/shards/shard_2560000_2712385")

for j, row in result.iterrows():
    print(row["idx"])
    print(row["distances"])
    print(row["neighbour_idxs"])

    if j >= 10:
        break

2560000
[0.99999994 0.99999994 0.99999994 0.99999994 0.99999994 0.99999994
 0.99999994 0.99999994 0.99999994 0.99999994 0.99999994 0.99999994
 0.99999994 0.99999994 0.99999994 0.99999994 0.99999994 0.99999994
 0.99999994 0.99999994 0.99999994 0.99999994 0.99999994 0.99999994
 0.99999994 0.99999994 0.99999994 0.99999994 0.99999994 0.99999994
 0.99999994 0.99999994 0.99999994 0.99999994 0.99999994 0.99999994
 0.99999994 0.99999994 0.99999994 0.99999994 0.99999994 0.9690302
 0.96692467 0.9626635  0.9624096  0.9598106  0.9598106  0.9598106
 0.9598106  0.9598106  0.9585372  0.95823973 0.9567757  0.9567757
 0.9567757  0.9567757  0.9567757  0.9567757  0.9567757  0.9567757
 0.9567757  0.9567757  0.9567757  0.9567757  0.9567757  0.9567757
 0.95677567 0.95677567 0.95677567 0.95677567 0.95677567 0.95677567
 0.95677567 0.95677567 0.95677567 0.95677567 0.95677567 0.95677567
 0.95677567 0.95677567 0.95677567 0.95677567 0.95677567 0.95677567
 0.95677567 0.95677567 0.95677567 0.95677567 0.95677567 0.9

In [21]:
pd.read_parquet("dumps/production/i2i/name_area/shards/shard_2560000_2712385").head(5)

,idx,distances,neighbour_idxs
0,2560000,"[0.99999994, 0.99999994, 0.99999994, 0.9999999...","[2706803, 2560000, 2520887, 2496353, 2311168, ..."
1,2560001,"[0.9999999, 0.99506056, 0.99506056, 0.99506056...","[2560001, 2447659, 1510305, 750207, 780917, 20..."
2,2560002,"[0.9999999, 0.9999999, 0.9999999, 0.9999999, 0...","[2560002, 1778998, 1254286, 812459, 530273, 28..."
3,2560003,"[1.0, 0.9588423, 0.9536178, 0.9528687, 0.95286...","[2560003, 848589, 1848735, 2652458, 2516340, 2..."
4,2560004,"[1.0, 0.9619129, 0.9560746, 0.95606756, 0.9557...","[2560004, 1588837, 2678024, 1835346, 1538212, ..."


In [22]:
shards_pathes = list(map(lambda x: path_join(shards_root, x), os.listdir(shards_root)))
shards_pathes

['dumps/production/i2i/name_area/shards/shard_768000_1024000',
 'dumps/production/i2i/name_area/shards/shard_2304000_2560000',
 'dumps/production/i2i/name_area/shards/shard_1792000_2048000',
 'dumps/production/i2i/name_area/shards/shard_0_256000',
 'dumps/production/i2i/name_area/shards/shard_512000_768000',
 'dumps/production/i2i/name_area/shards/shard_2560000_2712385',
 'dumps/production/i2i/name_area/shards/shard_1024000_1280000',
 'dumps/production/i2i/name_area/shards/shard_1536000_1792000',
 'dumps/production/i2i/name_area/shards/shard_1280000_1536000',
 'dumps/production/i2i/name_area/shards/shard_2048000_2304000',
 'dumps/production/i2i/name_area/shards/shard_256000_512000']

In [24]:
vacancies = pd.read_parquet(vacancies_tmp_path)

print(vacancies.head(3))

vacancy_ids = vacancies["vacancy_id"].to_list()
idx_to_vacancy_id = {it[0]: it[1] for it in enumerate(vacancy_ids)}
idx_to_vacancy_id[100]

  vacancy_id                                               name
0   v_862116          Чеченская республика. Смотритель музейный
1   v_288642  Самарская область. Ведущий менеджер по работе ...
2  v_1840054   Санкт-Петербург. Бухгалтер (по расчету зарплаты)


'v_544753'

In [26]:
output_i2i_df_path = path_join(embedding_root, "i2i.parquet")

In [33]:
result_vacancy_ids = []
result_neighbours  = []

for shard_path in shards_pathes:
    i2i_shard = pd.read_parquet(shard_path)

    for row_index, row in tqdm(i2i_shard.iterrows(), total=i2i_shard.shape[0]):
        vacancy_id = idx_to_vacancy_id[row["idx"]]
        neighbours = map(lambda x: x[0], filter(lambda x: x[1] >= 0.9, zip(row["neighbour_idxs"], row["distances"])))
        neighbours = map(lambda i: idx_to_vacancy_id[i], neighbours)
        neighbours = list(filter(lambda neighbour: neighbour != vacancy_id, neighbours))[:100]

        result_vacancy_ids.append(vacancy_id)
        result_neighbours.append(neighbours)

result_df = pd.DataFrame.from_dict({
    "vacancy_id": result_vacancy_ids,
    "neighbours": result_neighbours
})

result_df.to_parquet(output_i2i_df_path)
result_df.head(10)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 256000/256000 [01:05<00:00, 3884.08it/s]


,vacancy_id,neighbours
0,v_210485,"[v_1579639, v_2438371, v_463998, v_1655196, v_..."
1,v_1414033,"[v_1436818, v_1706535, v_1183479, v_2145514, v..."
2,v_447443,"[v_2589185, v_1843224, v_1461488, v_1817544, v..."
3,v_2419140,[]
4,v_1157009,"[v_918944, v_306770, v_685415, v_758707, v_115..."
5,v_2257180,"[v_811259, v_87818, v_1713003, v_1637499, v_25..."
6,v_763762,"[v_1514206, v_1231479, v_668006, v_409922, v_1..."
7,v_1919641,"[v_817490, v_2019692, v_550401, v_2285702, v_7..."
8,v_224700,"[v_1617218, v_892057, v_3683, v_334270, v_2563..."
9,v_1101679,"[v_1221089, v_777412, v_1238831, v_470564, v_3..."


In [39]:
output_i2i_df_path

'dumps/production/i2i/name_area/i2i.parquet'

In [36]:
# result_df.sample(n=50000).to_parquet(output_i2i_df_path)

In [34]:
result_df.shape[0]

2712385

In [41]:
result_df["neighbours"].map(len).max()

100

In [35]:
names = vacancies["name"].to_list()
idx_to_vacancy_id = {it[0]: it[1] for it in zip(vacancy_ids, names)}
idx_to_vacancy_id["v_301038"]

# 'Заведующий аптекой/Директор аптеки'

'Московская область. Заведующий аптекой/Директор аптеки'

In [38]:
for row_idx, row in result_df.iterrows():
    print("-=-=-=-=-=-=-==-=-=")
    print(idx_to_vacancy_id[row["vacancy_id"]])

    for neighbour_idx, neighbour in enumerate(row["neighbours"]):

        print("  ", idx_to_vacancy_id[neighbour])

        if neighbour_idx >= 3:
            break

    if row_idx >= 10:
        break

-=-=-=-=-=-=-==-=-=
Красноярский край. Водитель автобуса
   Красноярский край. Водитель автобуса
   Красноярский край. Водитель автобуса
   Красноярский край. Водитель автобуса
   Красноярский край. Водитель автобуса
-=-=-=-=-=-=-==-=-=
Пермский край. Курьер, подработка
   Пермский край. Курьер, подработка
   Пермский край. Курьер, подработка
   Пермский край. Курьер, подработка
   Пермский край. Курьер, на подработку
-=-=-=-=-=-=-==-=-=
Томская область. Торговый агент
   Томская область. Торговый агент
   Томская область. Торговый агент
   Томская область. Торговый агент на комбинированных маршрутах
   Томская область. Торговый представитель
-=-=-=-=-=-=-==-=-=
Рязанская область. Инженер технической поддержки пользователей со знанием английского языка
-=-=-=-=-=-=-==-=-=
Москва. Кассир (Москва, Осенний, 12)
   Москва. Кассир (Москва, Декабристов, 12)
   Москва. Пекарь (Москва, Осенний, 12)
   Москва. Пекарь (Москва, Осенний, 12)
   Москва. Кассир (Москва, Островитянова, 7)
-=-=-=-=-=-

In [51]:
result_df = pd.read_parquet(output_i2i_df_path)